# Save, Load and Export Models in Keras

## Task 1: Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import os

print('This notebook works with TensorFlow version:', tf.__version__)

folders = ['tmp', 'models', 'model_name', 'weights']
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

print(os.listdir('.'))

This notebook works with TensorFlow version: 2.0.1
['.ipynb_checkpoints', 'models', 'model_name', 'Student Notebook.ipynb', 'tmp', 'weights']


## Task 2: Create Model

In [4]:
def create_model():
    # 1st layer is mnist dataset
    # 3rd layer is output layer with 10 classes
    model = tf.keras.models.Sequential([tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
                                      tf.keras.layers.Dense(128, activation='relu'),
                                      tf.keras.layers.Dense(10,activation='softmax')])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model


In [5]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Task 3: Data Preprocessing

In [6]:
# load the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = np.reshape(x_train, (x_train.shape[0], 784))/255.
x_test = np.reshape(x_test, (x_test.shape[0], 784))/255.

# one-hot encoding the lables
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)


## Task 4: Model Checkpoint During Training

In [9]:
# We can save model weights using model checkpoint callbacks
checkpoint_dir = 'weights/'
epochs=25
batch_size=512
# callbacks allow us to do model checkpoint during training
# Weights are saved as .h5 format
callbacks=[tf.keras.callbacks.ModelCheckpoint(os.path.join(checkpoint_dir,'epoch_{epoch:02d}_acc_{val_acc:.2f}'),monitor='val_acc', save_weights_only=True, save_best_only=True)]
# save_weights_only saves model weights only and not model architecture
_ = model.fit(x_train,y_train, validation_data=(x_test,y_test), epochs=epochs, batch_size=batch_size, callbacks=callbacks)


Train on 60000 samples, validate on 10000 samples
Epoch 1/25
60000/60000 [==============================] - 7s 112us/sample - loss: 0.6938 - acc: 0.7637 - val_loss: 0.5052 - val_acc: 0.8182
Epoch 2/25
60000/60000 [==============================] - 2s 39us/sample - loss: 0.4352 - acc: 0.8468 - val_loss: 0.4473 - val_acc: 0.8385
Epoch 3/25
60000/60000 [==============================] - 3s 44us/sample - loss: 0.3876 - acc: 0.8628 - val_loss: 0.4124 - val_acc: 0.8505
Epoch 4/25
60000/60000 [==============================] - 3s 46us/sample - loss: 0.3542 - acc: 0.8734 - val_loss: 0.3918 - val_acc: 0.8612
Epoch 5/25
60000/60000 [==============================] - 3s 42us/sample - loss: 0.3317 - acc: 0.8804 - val_loss: 0.3613 - val_acc: 0.8713
Epoch 6/25
60000/60000 [==============================] - 2s 41us/sample - loss: 0.3141 - acc: 0.8867 - val_loss: 0.3627 - val_acc: 0.8688
Epoch 7/25
60000/60000 [==============================] - 3s 42us/sample - loss: 0.3056 - acc: 0.8899 - val_loss: 0

In [10]:
os.listdir(checkpoint_dir)

['checkpoint',
 'epoch_01_acc_0.82.data-00000-of-00001',
 'epoch_01_acc_0.82.index',
 'epoch_02_acc_0.84.data-00000-of-00001',
 'epoch_02_acc_0.84.index',
 'epoch_03_acc_0.85.data-00000-of-00001',
 'epoch_03_acc_0.85.index',
 'epoch_04_acc_0.86.data-00000-of-00001',
 'epoch_04_acc_0.86.index',
 'epoch_05_acc_0.87.data-00000-of-00001',
 'epoch_05_acc_0.87.index',
 'epoch_07_acc_0.87.data-00000-of-00001',
 'epoch_07_acc_0.87.index',
 'epoch_09_acc_0.88.data-00000-of-00001',
 'epoch_09_acc_0.88.index',
 'epoch_10_acc_0.88.data-00000-of-00001',
 'epoch_10_acc_0.88.index',
 'epoch_12_acc_0.88.data-00000-of-00001',
 'epoch_12_acc_0.88.index',
 'epoch_18_acc_0.89.data-00000-of-00001',
 'epoch_18_acc_0.89.index',
 'epoch_20_acc_0.89.data-00000-of-00001',
 'epoch_20_acc_0.89.index']

## Task 5: Load Weights

In [12]:
# Create a blank instance [random weights]->untrained model
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.3604580329895017, 0.1266]


In [14]:
# Don't load the data file, load the index file and remove .index from the path
model.load_weights('weights/epoch_20_acc_0.89')
print(model.evaluate(x_test, y_test, verbose=False))

[0.3184475895166397, 0.8908]


## Task 6: Saving Complete Model During Training

In [15]:
checkpoint_model_dir = 'models/'
epochs=25
batch_size=512

# Let's initialize a fresh instance of the model [untrained model]
model=create_model()
# Weights can be saved in any format, preferrably .h5
callbacks=[tf.keras.callbacks.ModelCheckpoint(os.path.join(checkpoint_model_dir,
                                                           'epoch_{epoch:02d}_acc_{val_acc:.2f}.h5'),
                                              monitor='val_acc', 
                                              save_weights_only=False, 
                                              save_best_only=True)]
# save_weights_only=False saves model weights with model architecture
_ = model.fit(x_train,y_train, validation_data=(x_test,y_test), epochs=epochs, batch_size=batch_size, callbacks=callbacks)


Train on 60000 samples, validate on 10000 samples
Epoch 1/25
60000/60000 [==============================] - 3s 46us/sample - loss: 0.6939 - acc: 0.7688 - val_loss: 0.4891 - val_acc: 0.8251
Epoch 2/25
60000/60000 [==============================] - 2s 37us/sample - loss: 0.4295 - acc: 0.8501 - val_loss: 0.4301 - val_acc: 0.8457
Epoch 3/25
60000/60000 [==============================] - 2s 38us/sample - loss: 0.3885 - acc: 0.8621 - val_loss: 0.4063 - val_acc: 0.8541
Epoch 4/25
60000/60000 [==============================] - 2s 37us/sample - loss: 0.3596 - acc: 0.8713 - val_loss: 0.3952 - val_acc: 0.8566
Epoch 5/25
60000/60000 [==============================] - 2s 36us/sample - loss: 0.3412 - acc: 0.8773 - val_loss: 0.3685 - val_acc: 0.8673
Epoch 6/25
60000/60000 [==============================] - 2s 35us/sample - loss: 0.3238 - acc: 0.8833 - val_loss: 0.3566 - val_acc: 0.8725
Epoch 7/25
60000/60000 [==============================] - 2s 36us/sample - loss: 0.3072 - acc: 0.8895 - val_loss: 0.

In [16]:
os.listdir(checkpoint_model_dir)

['epoch_01_acc_0.83.h5',
 'epoch_02_acc_0.85.h5',
 'epoch_03_acc_0.85.h5',
 'epoch_04_acc_0.86.h5',
 'epoch_05_acc_0.87.h5',
 'epoch_06_acc_0.87.h5',
 'epoch_08_acc_0.87.h5',
 'epoch_09_acc_0.88.h5',
 'epoch_10_acc_0.88.h5',
 'epoch_15_acc_0.89.h5',
 'epoch_18_acc_0.89.h5',
 'epoch_21_acc_0.89.h5',
 'epoch_22_acc_0.89.h5']

## Task 7: Load Models

In [19]:
# Create a fresh instance
model=create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.475870908355713, 0.0531]


In [21]:
model=tf.keras.models.load_model('models/epoch_22_acc_0.89.h5')
print(model.evaluate(x_test, y_test, verbose=False))

[0.32165690634250643, 0.8908]


In [22]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_11 (Dense)             (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Task 8: Manually Saving Weights and Models

In [24]:
# Saving just the weights
model.save_weights('tmp/manually_saved.w')
os.listdir('tmp')

['checkpoint',
 'manually_saved.w.data-00000-of-00001',
 'manually_saved.w.index']

In [25]:
# Saving an entire model
model.save('tmp/manually_saved.h5')
os.listdir('tmp')

['checkpoint',
 'manually_saved.h5',
 'manually_saved.w.data-00000-of-00001',
 'manually_saved.w.index']

## Task 9: Exporting and Restoring SavedModel Format

In [26]:
# path name = model_name here
model.save('model_name')
os.listdir('model_name')

W0807 10:50:43.447345   640 deprecation.py:506] From c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


['assets', 'saved_model.pb', 'variables']

In [27]:
# Bringing in the already saved model into keras-model
model = tf.keras.models.load_model('model_name')
print(model.evaluate(x_test,y_test, verbose=False))

[0.32114194617271424, 0.8908]
